# 개요


- keras를 이용하여 CNN을 구현한다.
- mnist 데이터 사용
- 모델을 학습 => 덤프 => site 구축(Flask) => 이미지를 업로드 => 서버에서 예측 => 결과를 돌려준다.
- keras는 딥러닝 엔진을 백엔드로 두고 그 위에서(하이레벨에서) 구동되는 인터페이스다. => 고수준 API를 이용한 딥러닝
  - 텐서플로는 저수준 API를 이용한 딥러닝이라고 이야기할 수 있다.
  - keras는 텐서플로가 아닌 다른 엔진도 쓸 수 있다.
- 난이도는 딥러닝 쪽에서는 가장 쉽다.

# 기본 모듈 가져오기

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [2]:
from tensorflow import keras

In [3]:
# 텐서플로를 백엔드로 사용하는 keras로 CNN 구현하기
keras.__version__

'2.2.4-tf'

In [4]:
# GPU 사용
# 메모리 관련 세팅(차후) -> 디테일 설정
# 보편적으로는 메모리 관련 세팅 없이 사용
with tf.device('/device:GPU:0') :
  print("GPU 사용")

GPU 사용


# 난수 고정

- 목적 : 실험 환경을 일관성 있게 통제하여, 일관성 있는 결과를 내고, 요소별로 어떤 영향을 미치는지 점검할 때 유리

In [5]:
seed_value = 0

In [6]:
# os레벨
import os
os.environ['PYTHONHASHSEED'] = str(seed_value)

In [7]:
# 파이썬 기본 모듈 레벨
import random
random.seed(seed_value)

In [8]:
# numpy 레벨
import numpy as np
np.random.seed(seed_value)

In [9]:
# 텐서플로 레벨
tf.set_random_seed(seed_value)

In [10]:
# 케라스 레벨
from tensorflow.keras import backend
session_config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess           = tf.Session(graph=tf.get_default_graph(), config=session_config)
backend.set_session(sess)

# mnist 데이터 준비

In [11]:
from tensorflow.keras.datasets import mnist

In [12]:
# 데이터 로드
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [13]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [14]:
type(X_train), type(y_train), type(X_test), type(y_test)

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [15]:
# 데이터 형식 확인
# 케라스가 베이스로 두고 있는, 엔진의 데이터 포맷 확인
# 이 형식에 맞춰서 x를 정의
fmt = backend.image_data_format()
fmt

'channels_last'

In [16]:
# 이미지 세로
PIXEL_H   = X_train.shape[1]
# 이미지 가로
PIXEL_W   = X_train.shape[2] 
# 이미지 총 픽셀 수 => 이미지 1개당 총 특성 수
PIXEL     = PIXEL_H * PIXEL_W
LABEL_NUM = len(np.unique(y_train))

PIXEL_H, PIXEL_W, PIXEL, LABEL_NUM

(28, 28, 784, 10)

In [17]:
# 포맷에 따라 신경망의 입력 x의 형태를 맞출 수 있다.
input_channel = 1
if fmt == 'channels_last' :
  X_train     = X_train.reshape(-1, PIXEL_H, PIXEL_W, input_channel)
  X_test      = X_test.reshape(-1, PIXEL_H, PIXEL_W, input_channel)
  input_shape = ( PIXEL_H, PIXEL_W, input_channel )
elif fmt == 'channels_first' :
  X_train     = X_train.reshape(-1, input_channel, PIXEL_H, PIXEL_W)
  X_test      = X_test.reshape(-1, input_channel, PIXEL_H, PIXEL_W)
  input_shape = ( input_channel, PIXEL_H, PIXEL_W )

In [18]:
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [19]:
# 정규화를 위해서 타입을 정수에서 부동소수로 변경
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train.dtype, X_test.dtype

(dtype('float32'), dtype('float32'))

In [20]:
# 최대값 기준으로 정규화 수행했다.
X_train /= 255
X_test  /= 255

In [22]:
# 정답에 원-핫 인코딩 필요
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

y_train[:2], y_test[:2]

(array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32))

# 신경망 구성

- 스타일
  - 절차적 코드
  - 객체지향적 코드

In [23]:
# 모듈 가져오기
from tensorflow.keras import models, layers, backend

In [26]:
# 백엔드 정보(엔진, 이미지 포맷, 입실론값(임계값))
backend.backend(), backend.image_data_format(), backend.epsilon()

('tensorflow', 'channels_last', 1e-07)

In [27]:
# CNN 기준 -> 텐서플로로 앞에서 작성했던 방식을 그대로 차용하겠다.
# 1. 네트워크를 기반으로 하는 모델을 생성하겠다, 인공신경망을 만들겠다 선언
model= models.Sequential()

In [29]:
# 2. 입력층부터 시작하여 합성곱층 1F 구성
#    W값은 Conv2D 클래스의 기본 설정값으로 사용, b도 기본 설정값으로 사용
model.add(layers.Conv2D(
    filters     = 32,              # 출력 채널 수의 정수값
    kernel_size = (5,5),           # 커널의 크기
    strides     = (1,1),           # 스트라이드의 이동폭
    padding     = 'same',          # 패딩의 타입
    activation  = 'relu',          # 활성화 함수
    input_shape = input_shape      # 입력 데이터 순서
))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
# 3. 풀링층 1F 구성
model.add(layers.MaxPool2D())

In [ ]:
# 4. 합성곱층 2F 구성
model.add(layers.Conv2D())

In [ ]:
# 5. 풀링층 2F 구성
model.add(layers.MaxPool2D())

In [ ]:
# 6. 전결합층 구성
model.add(layers.Flatten())

In [ ]:
# 7. 과적합 방지층 구성
model.add(layers.Dropout())

In [ ]:
# 8. 출력층 구성
model.add(layers.Dense())

# 손실함수, 최적화, 학습, 예측

In [ ]:
model.compile()

# 실제 학습
- epoch, 조기학습종료 연결

In [28]:
# 몇 세대 학습할 것이냐
epochs     = 10     # 전체 데이터를 다 학습을 마치는 것이 1세대 => 10회 반복
batch_size = 128    # 1회 학습 시 128개 데이터를 가지고 학습을 수행 => 총 6만개가 끝나면 1 epoch 완료

In [ ]:
# 학습
with tf.device('/device:GPU:0') :
  hist = model.fit()

In [ ]:
# 시각화로 확인

In [ ]:
# 개선 => 조기학습 종료

In [ ]:
# 학습

In [ ]:
# 시각화로 확인

In [ ]:
# 모델 덤프 -> 서비스로 이동